In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
  .load("abfss://bronze@vvdatabricksstorage.dfs.core.windows.net/products")

In [0]:
display(df)

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
df.createOrReplaceTempView("products")

### **Functions**

In [0]:
%sql
Create or replace function databricks_cata.bronze.discount_func(p_price double)
returns double
language sql
return p_price * 0.90

In [0]:
%sql
select product_id, price, databricks_cata.bronze.discount_func(price) as discounted_price
from products

In [0]:
df = df.withColumn("discounted_price", expr("databricks_cata.bronze.discount_func(price)"))
display(df)

In [0]:
%sql
Create or replace function databricks_cata.bronze.uppper_func(p_brand String)
returns String
language python
as
$$
  return p_brand.upper()    
$$

In [0]:
%sql
select product_id, brand, databricks_cata.bronze.uppper_func(brand) as brand_upper
from products

### **Data Writing**

In [0]:
df.write.format("delta")\
  .mode("overwrite")\
  .option("path","abfss://silver@vvdatabricksstorage.dfs.core.windows.net/products").save()

In [0]:
%sql
create table if not exists databricks_cata.silver.products_silver
using delta
location "abfss://silver@vvdatabricksstorage.dfs.core.windows.net/products"